In [57]:
from carto.sql import SQLClient
from carto.auth import APIKeyAuthClient
import pandas as pd
from collections import defaultdict


In [11]:
USR_BASE_URL = "https://phl.carto.com/"
auth_client = APIKeyAuthClient(USR_BASE_URL,api_key=None)
sql = SQLClient(auth_client)

In [12]:
tax_deliq_query = """
           SELECT 
           street_address, 
           zip_code, 
           owner, 
           principal_due, 
           total_due, 
           payment_agreement, 
           num_years_owed, 
           most_recent_year_owed, 
           oldest_year_owed, 
           most_recent_payment_date, 
           total_assessment, 
           detail_building_description, 
           general_building_description, 
           building_category, 
           sheriff_sale, 
           lat, 
           lon 
           
           FROM 
           real_estate_tax_delinquencies 
           
           WHERE 
           zip_code=19125 and num_years_owed > 3 
           ORDER BY total_due
        """
data = sql.send(tax_deliq_query)


In [13]:
list_of_records = data['rows']
print(list_of_records)

[{'street_address': '02065 CORAL ST', 'zip_code': 19125, 'owner': 'CCD 105 LLC', 'principal_due': 0, 'total_due': 10.02, 'payment_agreement': 'false', 'num_years_owed': 7, 'most_recent_year_owed': 2019, 'oldest_year_owed': 1997, 'most_recent_payment_date': None, 'total_assessment': 15800, 'detail_building_description': 'VAC LAND RES < ACRE', 'general_building_description': 'vacantLand', 'building_category': 'residential', 'sheriff_sale': 'N', 'lat': 39.98186876, 'lon': -75.13187923}, {'street_address': '1242R LEOPARD ST', 'zip_code': 19125, 'owner': 'ANDREWS DAVID H', 'principal_due': 111.74000000000001, 'total_due': 158.72, 'payment_agreement': 'false', 'num_years_owed': 6, 'most_recent_year_owed': 2019, 'oldest_year_owed': 2014, 'most_recent_payment_date': '2013-05-15T00:00:00Z', 'total_assessment': 1200, 'detail_building_description': 'VAC LAND RES < ACRE', 'general_building_description': 'vacantLand', 'building_category': 'residential', 'sheriff_sale': 'N', 'lat': 39.96992403, 'lon

In [63]:
def clean_leading_zero(address):
    while address.startswith('0'):
        address = address[1:]
    return address


location_dict = {}
for record in list_of_records:
#     location_dict[clean_leading_zero(record['street_address'])] = record
    record['clean_location'] = clean_leading_zero(record['street_address'])
    


### Export to csv for mapping

In [ ]:
# new = df[['street_address', 'total_due', 'lat', 'lon']].copy()
# new.to_csv('testing.csv', index=False)

In [ ]:

# df['cleaned_street_address'] = df['street_address'].apply(clean_leading_zero)



In [ ]:
# df

In [54]:
def prop_data(location):
    prop_query = '''
             SELECT
             location,
             building_code_description,
             category_code_description,
             depth,
             frontage,
             owner_1,
             owner_2,
             sale_date,
             sale_price,
             year_built,
             total_area
             
             FROM
             opa_properties_public
             
             WHERE
             location like '%{}%'
             '''

    response = sql.send(prop_query.format(location))
    return response['rows']



In [55]:
# df.cleaned_street_address.apply(lambda x: pd.Series(prop_data(x)))

In [65]:
for row in list_of_records:
    key = row['clean_location']
    print(key)
    result = prop_data(key)
    print(len(result))
    
    if len(result) >=1:
        row.update(result[0])
        print(row)

2065 CORAL ST
1
{'street_address': '02065 CORAL ST', 'zip_code': 19125, 'owner': 'CCD 105 LLC', 'principal_due': 0, 'total_due': 10.02, 'payment_agreement': 'false', 'num_years_owed': 7, 'most_recent_year_owed': 2019, 'oldest_year_owed': 1997, 'most_recent_payment_date': None, 'total_assessment': 15800, 'detail_building_description': 'VAC LAND RES < ACRE', 'general_building_description': 'vacantLand', 'building_category': 'residential', 'sheriff_sale': 'N', 'lat': 39.98186876, 'lon': -75.13187923, 'location': '2065 CORAL ST', 'building_code_description': 'VACANT LAND RESIDE < ACRE', 'category_code_description': 'Vacant Land', 'depth': 62.63, 'frontage': 12, 'owner_1': 'CCD 105 LLC              ', 'owner_2': None, 'sale_date': '2019-10-04T00:00:00Z', 'sale_price': 102500, 'year_built': '0000', 'total_area': 751.5600000000001, 'clean_location': '2065 CORAL ST'}
1242R LEOPARD ST
1
{'street_address': '1242R LEOPARD ST', 'zip_code': 19125, 'owner': 'ANDREWS DAVID H', 'principal_due': 111.74

1
{'street_address': '01935 E LETTERLY ST', 'zip_code': 19125, 'owner': 'JAMES JOHN ANTHONY III', 'principal_due': 153.75, 'total_due': 917.6800000000001, 'payment_agreement': 'true', 'num_years_owed': 5, 'most_recent_year_owed': 2019, 'oldest_year_owed': 2015, 'most_recent_payment_date': '2019-10-24T00:00:00Z', 'total_assessment': 38900, 'detail_building_description': 'ROW 2 STY MASONRY', 'general_building_description': 'house', 'building_category': 'residential', 'sheriff_sale': 'N', 'lat': 39.98569607, 'lon': -75.12866622, 'location': '1935 E LETTERLY ST', 'building_code_description': 'ROW 2 STY MASONRY', 'category_code_description': 'Single Family', 'depth': 50.25, 'frontage': 12, 'owner_1': 'JAMES JOHN ANTHONY III   ', 'owner_2': None, 'sale_date': '2013-02-15T00:00:00Z', 'sale_price': 5000, 'year_built': '1875', 'total_area': 603, 'clean_location': '1935 E LETTERLY ST'}
2133 E HUNTINGDON ST
1
{'street_address': '2133 E HUNTINGDON ST', 'zip_code': 19125, 'owner': 'ARITAN MURAT', '

1
{'street_address': '1806 E OAKDALE ST', 'zip_code': 19125, 'owner': 'KENSINGTON 2035 LLC', 'principal_due': 134.38, 'total_due': 1372.18, 'payment_agreement': 'false', 'num_years_owed': 22, 'most_recent_year_owed': 2019, 'oldest_year_owed': 1997, 'most_recent_payment_date': '2018-06-07T00:00:00Z', 'total_assessment': 9600, 'detail_building_description': 'VAC LAND RES < ACRE', 'general_building_description': 'vacantLand', 'building_category': 'residential', 'sheriff_sale': 'N', 'lat': 39.98922712, 'lon': -75.12585282, 'location': '1806 E OAKDALE ST', 'building_code_description': 'VACANT LAND RESIDE < ACRE', 'category_code_description': 'Vacant Land', 'depth': 57.5, 'frontage': 13, 'owner_1': 'KENSINGTON 2035 LLC      ', 'owner_2': None, 'sale_date': '2015-11-13T00:00:00Z', 'sale_price': 1500, 'year_built': '0000', 'total_area': 747.5, 'clean_location': '1806 E OAKDALE ST'}
2310 MEMPHIS ST
1
{'street_address': '2310 MEMPHIS ST', 'zip_code': 19125, 'owner': 'WALLACE LORETTA A', 'princip

1
{'street_address': '2545 COLLINS ST', 'zip_code': 19125, 'owner': 'OLIVER LILLIAN', 'principal_due': 850.23, 'total_due': 1771.52, 'payment_agreement': 'false', 'num_years_owed': 14, 'most_recent_year_owed': 2019, 'oldest_year_owed': 2006, 'most_recent_payment_date': '2019-10-18T00:00:00Z', 'total_assessment': 6000, 'detail_building_description': 'VAC LAND RES < ACRE', 'general_building_description': 'vacantLand', 'building_category': 'residential', 'sheriff_sale': 'N', 'lat': 39.98388462, 'lon': -75.12425803, 'location': '2545 COLLINS ST', 'building_code_description': 'VACANT LAND RESIDE < ACRE', 'category_code_description': 'Vacant Land', 'depth': 37, 'frontage': 12.16, 'owner_1': 'OLIVER LILLIAN M TRS', 'owner_2': None, 'sale_date': '1972-11-30T00:00:00Z', 'sale_price': 10500, 'year_built': '0000', 'total_area': 458.38, 'clean_location': '2545 COLLINS ST'}
1480 E MONTGOMERY AVE
1
{'street_address': '1480 E MONTGOMERY AVE', 'zip_code': 19125, 'owner': 'PAXON JOSEPH ETAL', 'principa

1
{'street_address': '305 E LEHIGH AVE', 'zip_code': 19125, 'owner': 'GARNICA JAVIER', 'principal_due': 1073.17, 'total_due': 2202.86, 'payment_agreement': 'true', 'num_years_owed': 6, 'most_recent_year_owed': 2019, 'oldest_year_owed': 2014, 'most_recent_payment_date': '2019-10-15T00:00:00Z', 'total_assessment': 70000, 'detail_building_description': 'ROW 3 STY MASONRY', 'general_building_description': 'house', 'building_category': 'residential', 'sheriff_sale': 'N', 'lat': 39.99061992, 'lon': -75.12676009, 'location': '305 E LEHIGH AVE', 'building_code_description': 'ROW 3 STY MASONRY', 'category_code_description': 'Single Family', 'depth': 68, 'frontage': 15, 'owner_1': 'GARNICA JAVIER           ', 'owner_2': None, 'sale_date': '1996-02-19T00:00:00Z', 'sale_price': 1, 'year_built': '1935', 'total_area': 1020, 'clean_location': '305 E LEHIGH AVE'}
1809 E BERKS ST
1
{'street_address': '01809 E BERKS ST', 'zip_code': 19125, 'owner': '1809 E BERKS ST LLC', 'principal_due': 1732.8500000000

...

1
{'street_address': '02134 E HAGERT ST', 'zip_code': 19125, 'owner': 'NELSON WRIGHT L', 'principal_due': 7191.06, 'total_due': 17384.72, 'payment_agreement': 'true', 'num_years_owed': 20, 'most_recent_year_owed': 2019, 'oldest_year_owed': 1998, 'most_recent_payment_date': '2019-10-10T00:00:00Z', 'total_assessment': 81400, 'detail_building_description': 'ROW 3 STY MASONRY', 'general_building_description': 'house', 'building_category': 'residential', 'sheriff_sale': 'N', 'lat': 39.98193058, 'lon': -75.12657492, 'location': '2134 E HAGERT ST', 'building_code_description': 'ROW 3 STY MASONRY', 'category_code_description': 'Single Family', 'depth': 41, 'frontage': 12.16, 'owner_1': 'NELSON WRIGHT L          ', 'owner_2': None, 'sale_date': '1999-12-15T00:00:00Z', 'sale_price': 9000, 'year_built': '1875', 'total_area': 498.56, 'clean_location': '2134 E HAGERT ST'}
1019 E EYRE ST
1
{'street_address': '1019 E EYRE ST', 'zip_code': 19125, 'owner': 'BELL HANGER COMPANY', 'principal_due': 9661.2

1
{'street_address': '1117 MARLBOROUGH ST', 'zip_code': 19125, 'owner': 'WILLIAMS FRANK', 'principal_due': 12303.58, 'total_due': 20484.13, 'payment_agreement': 'true', 'num_years_owed': 7, 'most_recent_year_owed': 2017, 'oldest_year_owed': 2010, 'most_recent_payment_date': '2019-10-10T00:00:00Z', 'total_assessment': 290400, 'detail_building_description': 'SEMI/DET 3 STY MASON', 'general_building_description': 'house', 'building_category': 'residential', 'sheriff_sale': 'N', 'lat': 39.96912928, 'lon': -75.13116894, 'location': '1117 MARLBOROUGH ST', 'building_code_description': 'SEMI/DET 3 STY MASONRY', 'category_code_description': 'Single Family', 'depth': 90, 'frontage': 20, 'owner_1': 'WILLIAMS FRANK J         ', 'owner_2': 'MARGARET E               ', 'sale_date': '1958-05-09T00:00:00Z', 'sale_price': 1, 'year_built': '1915', 'total_area': 1800, 'clean_location': '1117 MARLBOROUGH ST'}
2457 SEPVIVA ST
1
{'street_address': '2457 SEPVIVA ST', 'zip_code': 19125, 'owner': 'SHAHAJ SHERI

1
{'street_address': '449 E WILDEY ST', 'zip_code': 19125, 'owner': 'MARTIN JOHN', 'principal_due': 15738.85, 'total_due': 22759.36, 'payment_agreement': 'true', 'num_years_owed': 7, 'most_recent_year_owed': 2019, 'oldest_year_owed': 2010, 'most_recent_payment_date': '2016-07-13T00:00:00Z', 'total_assessment': 313200, 'detail_building_description': 'ROW  3STY MAS.+OTHER', 'general_building_description': 'house', 'building_category': 'residential', 'sheriff_sale': 'N', 'lat': 39.96976547, 'lon': -75.12848613, 'location': '449 E WILDEY ST', 'building_code_description': 'ROW 3 STY MASONRY+OTHER', 'category_code_description': 'Single Family', 'depth': 82.44, 'frontage': 13.620000000000001, 'owner_1': 'JOHN H MARTIN            ', 'owner_2': 'MARY                     ', 'sale_date': '1967-08-15T00:00:00Z', 'sale_price': 1, 'year_built': '1920', 'total_area': 1122.83, 'clean_location': '449 E WILDEY ST'}
1045-49 SARAH ST
1
{'street_address': '1045-49 SARAH ST', 'zip_code': 19125, 'owner': 'AN

1
{'street_address': '02461 GAUL ST', 'zip_code': 19125, 'owner': 'HYDUK WALTER', 'principal_due': 16583.96, 'total_due': 27960.440000000002, 'payment_agreement': 'false', 'num_years_owed': 14, 'most_recent_year_owed': 2019, 'oldest_year_owed': 2006, 'most_recent_payment_date': '2018-09-06T00:00:00Z', 'total_assessment': 158800, 'detail_building_description': 'ROW 2 STY MASONRY', 'general_building_description': 'house', 'building_category': 'residential', 'sheriff_sale': 'Y', 'lat': 39.97811852, 'lon': -75.12185271, 'location': '2461 GAUL ST', 'building_code_description': 'ROW 2 STY MASONRY', 'category_code_description': 'Single Family', 'depth': 60, 'frontage': 14, 'owner_1': 'HYDUK WALTER             ', 'owner_2': 'HYDUK MANDA              ', 'sale_date': '2014-05-13T00:00:00Z', 'sale_price': 1, 'year_built': '1925', 'total_area': 840, 'clean_location': '2461 GAUL ST'}
1358 E PALMER ST
1
{'street_address': '1358 E PALMER ST', 'zip_code': 19125, 'owner': 'SABATINO CAROL A', 'principal

1
{'street_address': '1340R-42 E COLUMBIA AVE', 'zip_code': 19125, 'owner': 'GOLDEN JOHN', 'principal_due': 25541.760000000002, 'total_due': 54104.56, 'payment_agreement': 'false', 'num_years_owed': 23, 'most_recent_year_owed': 2019, 'oldest_year_owed': 1997, 'most_recent_payment_date': None, 'total_assessment': 257400, 'detail_building_description': 'ROW  2.5  STY  BRICK', 'general_building_description': 'house', 'building_category': 'residential', 'sheriff_sale': 'N', 'lat': 39.97247802, 'lon': -75.1320507, 'location': '1340R-42 E COLUMBIA AVE', 'building_code_description': 'ROW 2.5 STY MASONRY', 'category_code_description': 'Single Family', 'depth': 37.33, 'frontage': 32, 'owner_1': 'JOHN T GOLDEN            ', 'owner_2': 'MARY A                   ', 'sale_date': '1964-09-14T00:00:00Z', 'sale_price': 1, 'year_built': '1915', 'total_area': 1194.56, 'clean_location': '1340R-42 E COLUMBIA AVE'}
2124 E SUSQUEHANNA AVE
1
{'street_address': '2124 E SUSQUEHANNA AVE', 'zip_code': 19125, 'ow

In [66]:
print(list_of_records)

[{'street_address': '02065 CORAL ST', 'zip_code': 19125, 'owner': 'CCD 105 LLC', 'principal_due': 0, 'total_due': 10.02, 'payment_agreement': 'false', 'num_years_owed': 7, 'most_recent_year_owed': 2019, 'oldest_year_owed': 1997, 'most_recent_payment_date': None, 'total_assessment': 15800, 'detail_building_description': 'VAC LAND RES < ACRE', 'general_building_description': 'vacantLand', 'building_category': 'residential', 'sheriff_sale': 'N', 'lat': 39.98186876, 'lon': -75.13187923, 'location': '2065 CORAL ST', 'building_code_description': 'VACANT LAND RESIDE < ACRE', 'category_code_description': 'Vacant Land', 'depth': 62.63, 'frontage': 12, 'owner_1': 'CCD 105 LLC              ', 'owner_2': None, 'sale_date': '2019-10-04T00:00:00Z', 'sale_price': 102500, 'year_built': '0000', 'total_area': 751.5600000000001, 'clean_location': '2065 CORAL ST'}, {'street_address': '1242R LEOPARD ST', 'zip_code': 19125, 'owner': 'ANDREWS DAVID H', 'principal_due': 111.74000000000001, 'total_due': 158.72

In [62]:
import csv


In [68]:

keys = list_of_records[0].keys()
with open('tax_deliq_19125.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(list_of_records)